In [1]:
import glob, os
import pandas as pd
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

from collections import Counter

from sklearn.svm import SVR, SVC
from sklearn.model_selection import GridSearchCV
from sklearn.kernel_ridge import KernelRidge
from sklearn import  metrics

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score

import torch
import torch.nn as nn

In [2]:
from types import SimpleNamespace 
from typing import Tuple


class Dataset:
    def __init__(self, path):
        self.path = path

        dataset = dict(np.load(self.path, allow_pickle=True))
        
        self.X = dataset.pop('X')
        self.y = dataset.pop('y')
        self.meta = dataset.pop('meta')[()]
        
        assert len(dataset) == 0
        
        self.y_binary = np.where(self.y == None, 0, 1)
    
    def normalize(self, mean=None, std=None):
        # TODO: czy powinniśmy normalizować per współrzędna czy globalnie?
        if mean is None:
            mean = np.mean(self.X, axis=0)
            
        if std is None:
            std = np.std(self.X, axis=0, ddof=1)
            
        # NOTE: inplace operators to prevent memory allocations
        self.X -= mean
        self.X /= std
        
        return mean, std
    
    def sample(self, n, *, balanced=False, with_idx=False, random_state=None, x_with_frame=False) -> Tuple[np.ndarray, np.ndarray]:
        """
        Choice `n` random samples from dataset.
        
        @param n: number of random samples to choose,
        @param balanced: if True number of samples for each class will be aprox. equal,
        @param with_idx: return data indexes of sampled records,
        @param random_state: random state used to generate samples indices,
        """
        assert len(self.y.shape) == 1
        
        if balanced:
            counts = Counter(self.y)
            class_count = len(counts)
            
            # NOTE: https://stackoverflow.com/questions/35215161/most-efficient-way-to-map-function-over-numpy-array/35216364
            probs = np.array([1.0 / (class_count * counts[x]) for x in self.y])
        else:
            probs = None
            
        idx = np.random.RandomState(random_state).choice(self.y.size, size=n, p=probs)
        
        new_X = np.pad(self.X, pad_width=[(128, 128), (0, 0)], mode='edge') 
        
        if x_with_frame:
            X = np.array([new_X[index: index + 257] for index in idx])
        else:
            X = new_X[idx + 128]
        
        if with_idx:
            return idx, X, self.y[idx]
        
        return X, self.y[idx]
    
    def frame_to_time(self, frame: int) -> float:
        """Convert frame id to time in sec."""
        return librosa.core.frames_to_time(
            frame,
            sr=self.meta['sampling_rate'],
            hop_length=self.meta['hop_length'],
            n_fft=self.meta['n_fft']
        )

In [3]:
train_data = test_data = None
train_data = Dataset('ch1-2018-11-20_10-29-02_0000012.wav.trimed.npz')
print(train_data.path, Counter(train_data.y_binary))

mean, std = train_data.normalize()
print("mean: %f; std: %f" % (np.mean(train_data.X), np.std(train_data.X, ddof=1)))

#################################################################
print()
test_data = Dataset('ch1-2018-11-20_10-31-42_0000014.wav.trimed.npz')
print(test_data.path, Counter(test_data.y_binary))

mean, std = test_data.normalize()
print("mean: %f; std: %f" % (np.mean(test_data.X), np.std(test_data.X, ddof=1)))

ch1-2018-11-20_10-29-02_0000012.wav.trimed.npz Counter({0: 196372, 1: 38013})
mean: 0.000000; std: 1.000209

ch1-2018-11-20_10-31-42_0000014.wav.trimed.npz Counter({0: 146541, 1: 87844})
mean: 0.000000; std: 1.000337


In [4]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=5, stride=3))
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=5, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(8 * 8 * 256, 100)
        self.fc2 = nn.Linear(100, 9)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
    
    @property
    def n_param(self):
        return sum(p.numel() for p in self.parameters())

In [5]:
np.place(train_data.y, train_data.y == None, "None")
np.place(test_data.y, test_data.y == None, "None")


In [6]:
from sklearn.preprocessing import OrdinalEncoder
enc_train = OrdinalEncoder(dtype=int)
enc_train.fit(train_data.y.reshape(-1, 1))
enc_train.categories_

[array(['CMP', 'FL', 'FM', 'IU', 'None', 'RP', 'SH', 'ST', 'TR'],
       dtype=object)]

In [7]:
enc_test = OrdinalEncoder(dtype=int)
enc_test.fit(test_data.y.reshape(-1, 1))
enc_test.categories_

[array(['22kHz', 'CMP', 'FL', 'FM', 'None', 'RP', 'SH', 'ST', 'TR'],
       dtype=object)]

In [8]:

model = ConvNet()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Train the model

num_epochs = 20
total_step = 20
loss_list = []
acc_list = []
test_acc_list = []
for epoch in range(num_epochs):
    
    for i in range(total_step):
        images, labels = train_data.sample(50, balanced=True, with_idx=False, x_with_frame=True)
        images = torch.tensor(np.expand_dims(images, axis=1))
        labels = torch.tensor(enc_train.transform(labels.reshape(-1, 1)).reshape(-1), dtype=torch.long)
        
        # Run the forward pass
        outputs = model(images)
        
        loss = criterion(outputs, labels)
#         print(loss)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        print(predicted)
        print(labels)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)
        
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))
        
    # Test after epoch
    
    images, labels = test_data.sample(50, balanced=True, with_idx=False, x_with_frame=True)
    images = torch.tensor(np.expand_dims(images, axis=1))
    labels = torch.tensor(enc_test.transform(labels.reshape(-1, 1)).reshape(-1), dtype=torch.long)

    # Run the forward pass
    outputs = model(images)
    
    total = labels.size(0)
    _, predicted = torch.max(outputs.data, 1)
    print(predicted)
    print(labels)
    acc = (predicted == labels).sum().item() / total
    print("TEST ACC: ", acc )
    test_acc_list.append(acc)
    
    

tensor([2, 2, 5, 4, 5, 8, 5, 2, 5, 6, 5, 2, 2, 5, 5, 5, 5, 0, 5, 2, 5, 5, 5, 7,
        5, 2, 5, 5, 5, 2, 5, 5, 5, 2, 5, 5, 2, 6, 2, 2, 5, 5, 2, 5, 2, 5, 5, 5,
        5, 5])
tensor([2, 6, 5, 7, 0, 7, 1, 3, 5, 0, 6, 3, 6, 8, 1, 7, 1, 3, 3, 1, 1, 1, 8, 2,
        2, 3, 5, 0, 1, 1, 8, 2, 2, 5, 2, 7, 6, 1, 7, 1, 3, 1, 6, 4, 5, 7, 7, 8,
        5, 3])
Epoch [1/20], Step [1/20], Loss: 2.1818, Accuracy: 10.00%
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3])
tensor([5, 5, 4, 1, 2, 7, 8, 1, 5, 3, 6, 0, 0, 7, 1, 5, 5, 2, 5, 5, 6, 7, 2, 7,
        3, 8, 7, 7, 0, 8, 5, 8, 1, 5, 2, 0, 1, 8, 4, 1, 5, 7, 7, 0, 8, 6, 0, 0,
        6, 8])
Epoch [1/20], Step [2/20], Loss: 8.5867, Accuracy: 4.00%
tensor([2, 2, 2, 2, 2, 8, 2, 2, 2, 2, 8, 2, 8, 8, 8, 2, 2, 2, 8, 2, 8, 2, 2, 2,
        8, 8, 2, 2, 8, 8, 2, 2, 2, 8, 8, 2, 8, 8, 2, 2, 2, 2, 2, 2, 2, 8, 8, 8,
        2, 8])
tensor([7,

tensor([8, 7, 0, 6, 8, 7, 8, 8, 6, 8, 8, 3, 0, 8, 8, 3, 7, 7, 8, 7, 6, 0, 6, 8,
        8, 6, 7, 8, 6, 6, 8, 8, 8, 8, 8, 0, 0, 8, 0, 8, 7, 3, 8, 7, 6, 0, 8, 7,
        0, 8])
tensor([3, 7, 5, 5, 0, 2, 0, 0, 6, 3, 0, 3, 7, 3, 7, 6, 1, 1, 2, 2, 6, 0, 4, 5,
        0, 1, 7, 8, 4, 6, 6, 3, 0, 8, 3, 7, 7, 3, 5, 0, 5, 3, 2, 7, 5, 5, 0, 5,
        0, 0])
Epoch [2/20], Step [1/20], Loss: 1.9623, Accuracy: 24.00%
tensor([6, 7, 6, 8, 6, 7, 7, 8, 7, 7, 8, 8, 6, 7, 0, 3, 7, 6, 8, 7, 6, 8, 6, 6,
        7, 3, 8, 3, 3, 7, 0, 7, 7, 8, 8, 6, 8, 8, 6, 0, 0, 7, 0, 0, 7, 6, 8, 0,
        3, 7])
tensor([4, 6, 4, 2, 4, 6, 6, 8, 5, 4, 6, 3, 4, 7, 5, 4, 5, 5, 1, 5, 7, 0, 1, 1,
        4, 3, 8, 6, 3, 1, 8, 1, 1, 4, 3, 6, 3, 8, 1, 7, 2, 7, 7, 7, 1, 0, 8, 8,
        3, 6])
Epoch [2/20], Step [2/20], Loss: 1.9834, Accuracy: 20.00%
tensor([0, 7, 6, 8, 7, 8, 0, 0, 0, 7, 7, 8, 6, 0, 8, 7, 3, 0, 8, 3, 7, 0, 7, 8,
        0, 6, 8, 3, 0, 0, 6, 7, 6, 7, 3, 8, 3, 0, 7, 5, 6, 6, 8, 8, 7, 0, 0, 0,
        0, 0])
tensor([5

tensor([0, 0, 0, 0, 6, 6, 6, 6, 1, 0, 0, 6, 6, 3, 6, 6, 6, 0, 6, 0, 1, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 8, 1, 1, 0, 6, 2, 6, 3, 8, 6, 1, 1, 6, 6, 6,
        0, 3])
tensor([8, 7, 8, 0, 5, 6, 2, 5, 1, 8, 0, 5, 7, 3, 6, 5, 7, 2, 6, 7, 7, 7, 5, 6,
        5, 1, 4, 2, 5, 5, 5, 5, 4, 8, 2, 5, 0, 4, 2, 6, 4, 8, 2, 1, 1, 3, 7, 6,
        7, 3])
Epoch [3/20], Step [1/20], Loss: 1.8420, Accuracy: 34.00%
tensor([0, 0, 0, 6, 2, 6, 6, 6, 6, 6, 6, 6, 6, 1, 6, 6, 0, 3, 0, 0, 8, 6, 0, 6,
        1, 1, 8, 0, 1, 6, 6, 6, 6, 6, 0, 6, 6, 1, 6, 6, 0, 0, 6, 2, 1, 6, 6, 0,
        6, 0])
tensor([8, 8, 8, 1, 2, 2, 1, 1, 4, 4, 4, 5, 1, 5, 5, 0, 8, 3, 4, 2, 8, 2, 7, 1,
        5, 7, 5, 2, 2, 3, 6, 4, 6, 3, 8, 7, 4, 2, 7, 4, 2, 0, 4, 2, 5, 6, 4, 0,
        6, 0])
Epoch [3/20], Step [2/20], Loss: 1.9756, Accuracy: 22.00%
tensor([6, 6, 3, 1, 3, 6, 6, 6, 6, 6, 6, 1, 0, 1, 6, 3, 2, 8, 1, 1, 0, 1, 6, 6,
        6, 6, 6, 6, 1, 6, 6, 6, 6, 3, 1, 8, 1, 6, 0, 3, 5, 2, 6, 1, 1, 0, 6, 6,
        6, 6])
tensor([3

tensor([3, 1, 0, 1, 6, 6, 6, 6, 6, 0, 1, 6, 3, 6, 5, 2, 7, 0, 4, 3, 0, 8, 5, 0,
        5, 8, 6, 5, 5, 6, 3, 0, 6, 5, 8, 6, 8, 0, 1, 1, 6, 6, 1, 5, 6, 6, 7, 1,
        3, 1])
tensor([4, 1, 1, 1, 0, 6, 0, 8, 5, 2, 1, 5, 3, 2, 1, 7, 0, 0, 4, 3, 0, 8, 0, 2,
        1, 8, 4, 1, 4, 6, 3, 4, 6, 4, 0, 4, 7, 2, 4, 2, 4, 4, 5, 5, 7, 5, 1, 1,
        3, 5])
Epoch [4/20], Step [1/20], Loss: 1.6639, Accuracy: 34.00%
tensor([5, 3, 4, 6, 7, 5, 5, 5, 5, 0, 3, 1, 3, 5, 1, 0, 8, 0, 5, 8, 8, 3, 0, 5,
        6, 3, 6, 6, 6, 5, 6, 5, 6, 7, 6, 3, 5, 3, 1, 0, 6, 4, 7, 6, 3, 8, 6, 0,
        6, 4])
tensor([4, 3, 4, 6, 1, 4, 2, 5, 7, 8, 3, 2, 3, 6, 2, 2, 8, 2, 4, 8, 0, 6, 2, 5,
        2, 4, 4, 6, 6, 5, 2, 2, 1, 2, 8, 3, 2, 3, 1, 0, 2, 4, 0, 0, 3, 8, 6, 7,
        5, 6])
Epoch [4/20], Step [2/20], Loss: 1.5545, Accuracy: 40.00%
tensor([7, 6, 4, 5, 8, 6, 6, 5, 4, 7, 7, 6, 6, 6, 0, 6, 7, 5, 7, 6, 6, 7, 0, 5,
        7, 1, 8, 0, 6, 0, 6, 0, 3, 0, 4, 8, 4, 7, 7, 6, 5, 0, 5, 6, 0, 5, 4, 7,
        0, 1])
tensor([7

tensor([1, 2, 1, 4, 0, 1, 1, 7, 2, 1, 7, 3, 5, 6, 3, 6, 8, 8, 1, 3, 1, 5, 3, 4,
        8, 2, 8, 8, 7, 6, 6, 2, 8, 4, 6, 1, 6, 8, 5, 6, 6, 1, 8, 3, 2, 4, 2, 6,
        0, 1])
tensor([5, 2, 1, 1, 2, 6, 1, 7, 2, 4, 7, 3, 2, 1, 3, 6, 6, 8, 1, 3, 1, 6, 3, 4,
        7, 2, 8, 8, 7, 6, 4, 2, 8, 4, 5, 7, 1, 8, 2, 6, 6, 4, 5, 3, 2, 4, 2, 4,
        0, 1])
Epoch [5/20], Step [1/20], Loss: 1.1528, Accuracy: 64.00%
tensor([0, 4, 0, 3, 0, 6, 6, 1, 4, 2, 8, 0, 1, 2, 1, 5, 4, 6, 5, 5, 8, 6, 2, 5,
        6, 6, 6, 5, 4, 5, 0, 4, 6, 4, 0, 3, 3, 1, 3, 6, 2, 1, 7, 4, 2, 6, 8, 7,
        7, 4])
tensor([0, 2, 0, 3, 0, 6, 6, 7, 4, 7, 0, 0, 5, 2, 1, 2, 1, 6, 5, 5, 0, 2, 2, 5,
        0, 1, 2, 5, 4, 5, 2, 1, 7, 4, 7, 3, 3, 1, 3, 6, 1, 7, 7, 4, 2, 6, 8, 7,
        7, 6])
Epoch [5/20], Step [2/20], Loss: 1.1771, Accuracy: 62.00%
tensor([6, 8, 5, 6, 1, 1, 6, 1, 8, 6, 4, 3, 8, 6, 6, 8, 5, 1, 5, 0, 2, 0, 8, 6,
        7, 3, 6, 5, 6, 6, 5, 0, 4, 3, 6, 5, 4, 0, 2, 8, 7, 6, 2, 5, 5, 6, 2, 1,
        0, 2])
tensor([1

...

tensor([5, 4, 1, 7, 3, 1, 2, 6, 5, 3, 2, 4, 4, 8, 2, 0, 5, 5, 7, 6, 3, 3, 4, 0,
        3, 8, 5, 3, 2, 0, 1, 7, 4, 8, 0, 8, 6, 6, 2, 7, 3, 2, 8, 2, 8, 8, 3, 1,
        6, 5])
tensor([2, 1, 7, 7, 3, 7, 7, 6, 7, 3, 0, 4, 4, 4, 7, 0, 5, 5, 7, 5, 3, 0, 1, 8,
        3, 8, 5, 3, 7, 0, 1, 7, 4, 8, 0, 8, 8, 6, 2, 7, 3, 5, 8, 2, 8, 8, 3, 2,
        5, 5])
Epoch [9/20], Step [1/20], Loss: 1.0953, Accuracy: 64.00%
tensor([6, 6, 4, 6, 8, 4, 6, 4, 6, 0, 0, 3, 6, 1, 8, 0, 3, 2, 8, 1, 0, 1, 4, 4,
        7, 8, 8, 3, 5, 0, 5, 6, 3, 4, 6, 1, 5, 3, 6, 3, 3, 7, 8, 1, 7, 5, 4, 2,
        1, 7])
tensor([7, 6, 4, 2, 8, 4, 6, 4, 6, 0, 0, 3, 0, 2, 8, 0, 3, 5, 8, 1, 8, 7, 4, 4,
        7, 8, 4, 3, 2, 0, 5, 6, 3, 1, 6, 1, 7, 3, 6, 3, 3, 7, 8, 1, 7, 5, 4, 8,
        1, 7])
Epoch [9/20], Step [2/20], Loss: 0.9395, Accuracy: 76.00%
tensor([5, 8, 8, 6, 0, 5, 0, 4, 3, 8, 2, 8, 7, 5, 8, 3, 3, 3, 0, 8, 0, 2, 0, 0,
        3, 2, 2, 0, 6, 4, 2, 4, 6, 3, 0, 3, 8, 4, 7, 7, 3, 3, 4, 0, 1, 0, 1, 2,
        5, 0])
tensor([5

tensor([0, 8, 5, 3, 3, 7, 6, 3, 5, 2, 8, 7, 7, 4, 2, 8, 8, 6, 1, 1, 6, 5, 3, 1,
        2, 2, 6, 3, 6, 5, 7, 2, 4, 2, 3, 0, 7, 5, 5, 8, 8, 3, 0, 5, 3, 6, 2, 6,
        2, 7])
tensor([0, 4, 5, 3, 3, 7, 6, 3, 5, 2, 0, 7, 2, 4, 2, 2, 4, 6, 1, 1, 6, 6, 3, 1,
        2, 2, 6, 3, 6, 1, 7, 2, 4, 2, 3, 0, 7, 4, 5, 8, 8, 3, 8, 5, 3, 6, 2, 6,
        4, 7])
Epoch [10/20], Step [1/20], Loss: 0.7012, Accuracy: 80.00%
tensor([5, 6, 2, 2, 7, 3, 4, 3, 4, 4, 4, 0, 4, 7, 2, 1, 7, 3, 4, 6, 0, 1, 0, 8,
        6, 0, 7, 4, 2, 3, 4, 6, 1, 8, 2, 2, 8, 0, 4, 8, 2, 8, 8, 4, 5, 7, 4, 0,
        6, 3])
tensor([1, 6, 1, 1, 7, 3, 4, 3, 6, 4, 1, 0, 4, 0, 0, 2, 7, 3, 4, 6, 0, 1, 0, 8,
        5, 0, 7, 1, 2, 3, 4, 0, 4, 8, 2, 2, 8, 0, 4, 8, 0, 8, 8, 1, 5, 7, 6, 0,
        6, 3])
Epoch [10/20], Step [2/20], Loss: 0.8975, Accuracy: 70.00%
tensor([4, 2, 0, 3, 8, 6, 5, 7, 6, 3, 5, 6, 4, 1, 4, 2, 6, 7, 4, 1, 8, 4, 3, 4,
        1, 1, 2, 4, 3, 1, 1, 2, 4, 2, 5, 4, 6, 7, 2, 1, 4, 0, 1, 1, 3, 2, 3, 3,
        6, 3])
tensor(

tensor([0, 5, 1, 7, 1, 4, 1, 6, 0, 0, 4, 5, 2, 8, 5, 5, 2, 1, 7, 4, 3, 2, 4, 2,
        7, 4, 0, 0, 8, 4, 6, 3, 2, 8, 8, 7, 5, 0, 0, 8, 0, 3, 1, 0, 1, 4, 8, 7,
        0, 4])
tensor([8, 5, 1, 7, 1, 4, 2, 6, 0, 0, 8, 5, 1, 8, 5, 5, 2, 1, 7, 5, 3, 2, 4, 2,
        2, 4, 0, 7, 8, 4, 6, 3, 2, 8, 8, 7, 5, 0, 8, 8, 0, 3, 5, 8, 7, 4, 8, 7,
        0, 4])
Epoch [11/20], Step [1/20], Loss: 0.5371, Accuracy: 78.00%
tensor([6, 2, 6, 1, 4, 4, 0, 6, 7, 3, 6, 5, 1, 8, 8, 5, 5, 1, 4, 1, 4, 0, 1, 8,
        8, 1, 4, 1, 1, 3, 4, 1, 5, 1, 3, 5, 2, 4, 1, 3, 8, 0, 0, 3, 5, 6, 8, 8,
        7, 6])
tensor([6, 2, 6, 7, 4, 4, 0, 6, 7, 3, 6, 5, 7, 8, 8, 5, 7, 2, 4, 1, 4, 4, 1, 8,
        8, 2, 1, 1, 2, 3, 4, 2, 5, 1, 3, 5, 2, 4, 2, 3, 2, 0, 7, 1, 5, 6, 8, 8,
        1, 6])
Epoch [11/20], Step [2/20], Loss: 0.7849, Accuracy: 72.00%
tensor([6, 5, 3, 6, 0, 8, 5, 7, 1, 2, 6, 1, 0, 6, 3, 4, 3, 0, 6, 6, 4, 1, 1, 6,
        7, 3, 4, 2, 6, 0, 0, 6, 0, 5, 5, 1, 8, 4, 8, 7, 7, 5, 6, 5, 4, 3, 7, 4,
        3, 6])
tensor(

tensor([6, 0, 0, 0, 5, 0, 5, 2, 0, 2, 0, 7, 5, 3, 2, 1, 6, 1, 4, 7, 4, 5, 4, 6,
        7, 6, 1, 7, 5, 6, 2, 8, 6, 6, 7, 4, 5, 2, 2, 7, 0, 6, 8, 8, 0, 3, 8, 0,
        0, 7])
tensor([6, 0, 8, 0, 5, 0, 5, 2, 6, 5, 2, 7, 5, 3, 2, 1, 6, 7, 4, 7, 4, 5, 4, 6,
        7, 6, 1, 1, 5, 2, 2, 8, 6, 6, 7, 4, 5, 2, 2, 7, 0, 4, 8, 8, 0, 3, 8, 0,
        0, 7])
Epoch [12/20], Step [1/20], Loss: 0.4897, Accuracy: 84.00%
tensor([7, 7, 4, 8, 0, 7, 4, 2, 1, 0, 8, 7, 5, 3, 3, 0, 0, 0, 7, 5, 2, 6, 1, 2,
        1, 1, 8, 1, 1, 0, 6, 3, 2, 2, 1, 1, 7, 4, 5, 7, 3, 3, 6, 4, 6, 3, 1, 0,
        2, 2])
tensor([8, 4, 4, 8, 0, 7, 4, 2, 7, 4, 8, 7, 5, 3, 8, 2, 8, 7, 7, 5, 0, 6, 1, 2,
        1, 0, 8, 1, 1, 0, 6, 3, 2, 2, 1, 2, 1, 4, 5, 7, 3, 3, 6, 4, 6, 3, 1, 0,
        2, 2])
Epoch [12/20], Step [2/20], Loss: 0.7954, Accuracy: 76.00%
tensor([6, 5, 0, 3, 8, 3, 8, 8, 5, 5, 2, 6, 4, 5, 7, 3, 1, 8, 4, 5, 1, 2, 5, 8,
        5, 4, 0, 6, 0, 5, 3, 1, 1, 8, 3, 0, 0, 6, 3, 2, 4, 0, 3, 6, 5, 1, 6, 6,
        6, 0])
tensor(

tensor([2, 7, 0, 0, 3, 2, 3, 6, 4, 8, 1, 0, 5, 3, 7, 4, 3, 4, 6, 5, 3, 8, 6, 7,
        8, 0, 8, 2, 4, 1, 4, 7, 2, 2, 4, 8, 7, 3, 5, 3, 4, 2, 3, 4, 3, 8, 6, 8,
        2, 0])
tensor([2, 7, 0, 0, 3, 2, 3, 6, 4, 7, 1, 0, 5, 3, 7, 8, 3, 4, 6, 5, 3, 7, 6, 1,
        0, 0, 8, 2, 4, 6, 7, 4, 2, 0, 4, 8, 7, 3, 5, 3, 7, 2, 3, 1, 3, 8, 6, 8,
        2, 0])
Epoch [13/20], Step [1/20], Loss: 0.5935, Accuracy: 78.00%
tensor([3, 2, 8, 5, 0, 4, 7, 0, 2, 3, 6, 3, 0, 5, 0, 4, 0, 0, 7, 5, 2, 8, 7, 3,
        4, 6, 8, 2, 0, 6, 8, 6, 7, 3, 6, 6, 8, 1, 8, 4, 0, 0, 7, 1, 5, 8, 3, 4,
        7, 1])
tensor([3, 2, 8, 5, 0, 4, 1, 0, 2, 3, 6, 3, 0, 5, 0, 4, 0, 7, 7, 5, 2, 8, 7, 3,
        4, 6, 8, 2, 0, 6, 8, 6, 7, 3, 7, 6, 8, 1, 8, 4, 0, 0, 7, 2, 5, 8, 3, 4,
        7, 0])
Epoch [13/20], Step [2/20], Loss: 0.3847, Accuracy: 90.00%
tensor([8, 4, 6, 4, 3, 3, 4, 4, 8, 6, 2, 8, 0, 8, 3, 6, 4, 7, 8, 7, 2, 8, 4, 5,
        0, 1, 0, 8, 8, 3, 0, 5, 4, 2, 4, 4, 8, 5, 4, 5, 4, 6, 1, 8, 8, 6, 2, 2,
        3, 6])
tensor(

try:
- without equal sampling
- different test file
- larger train and/or test data